In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm, trange

In [3]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
referer = 'https://www.resilience.org/latest-articles/'
headers = {'User-Agent': user_agent, 'referer':referer}

In [16]:
topic_page = 'https://insideclimatenews.org/'
response = requests.get(topic_page, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
topics = [e.find('a')['href'] for e in soup.find('ul', {'id':'menu-main', 'class':'menu'}) \
           .find_all('li')]

In [17]:
a = topics[0]

In [18]:
a

'https://insideclimatenews.org/category/science/'

In [29]:
article_links = []
for topic in tqdm(topics):
    
    response = requests.get(topic, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    last_page = int(soup.find_all('a', {'class':'page-numbers'})[-2].text \
                    .replace("\n", "").replace(",", ""))

    for page in trange(1, last_page+1):
        url = topic + 'page/{}/'.format(page)

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        new_links = [e['href'] for e in soup.find('div', {'class':'posts-list'}).find_all('a')]
        article_links.extend(new_links)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

In [30]:
article_links_unique = list(set(article_links))
print(len(article_links))
print(len(article_links_unique))

6814
5742


In [52]:
def get_article_data(article):

    art_dict = {'url':article}
    
    response = requests.get(article, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        art_dict['title']    = soup.find('h1', {'class':'entry-title'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing title")
    
    try:
        art_dict['subtitle']    = soup.find('h2', {'class':'entry-subtitle'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing subtitle")

    try:
        author = soup.find('div', {'class':'byline'}).text.replace("\n", "")
        if author[:3] == "By ":
            author = author[3:]
        art_dict['author'] = author
    except:
        pass
        #print("Missing author")
    
    try:
        art_dict['date']    = soup.find('div', {'class':'date'}).text \
                                   .replace("\n", "")
    except:
        pass
        #print("Missing date")

    try:
        art_dict['text']    = " ".join([p.text for p in soup.find('div', {'class':'entry-content'}) \
                                                           .findChildren('p', recursive=False)])
    except:
        pass
        #print("Missing text")
    
    return art_dict

In [53]:
icn = []

for article in tqdm(article_links_unique):
    icn.append(get_article_data(article))

icn_df = pd.DataFrame(icn)

  0%|          | 0/5742 [00:00<?, ?it/s]

In [55]:
icn_df.to_pickle("../Data/Inside Climate News/icn.pkl")